# Dynamic Programing I

Dynamic Programing is a general technique to solve optimizaiton and/or search algorithms that allows to reduce the complexity from (usually) an exponential time complexity to a polinomial one.

The key insight is that the main problem can be divided into sub-problems which can be recursively solved. These problems have an **optimal** substructure.

In a sense, Dynamic Programing is a smart backtracking implementation.

Typical Dynamic Programing problems are counting or optimization ones.

A generic approach to solve this kind of problems is as follows:
1. Fund sub-problems.
2. "Guess" a partial solution.
3. Relate sub-problems.
4. Recursion and memoization, or use a bottom-up approach.
5. Find the solution to the original problem.

### 1. Find sub-problems

Divide the original problem into a number of related sub-problems.

For example: Let $S[n]$ be a solution to sub-problems using only the first $n$ constraints.

### 2. "Guess" partial solution

Construct a partial solution by "guessing" part of it.

For example: If we knew the optimal $n-1$ *-th* sub-problem solution, we can construct the *n-th* sub-problem solution as:

$$S[n] = S[n-1] \oplus c*(n)$$

### 3. Relate sub-problems

Find a general recursion for $S[n]$.

For example:

$$S[n] = \max_{i} S[n-1] \oplus c_i*(n)$$

and

$$S[1]=k$$

### 4. Recursion and memoization, or bottom-up approach.

Use recursion with memoization to solve the equation for $S[n]$ or build a solution $S[n]$ from the smalest patial solutions.

For example:

Memoization
```Python
@functools.lru_cache(None)
def S(n):
    return max(S(n-1) + c(i) for i in I)
```

Bottom-up
```Python
def S(n):
    partial_S = [CONST]
    for k in range(1, n+1):
        Sk = max(partial_S[k-1] + c(i) for i in I)
        partial_S.append(Sk)
    return partial_S[n]
```

### 5. Find the solution to the original problem.

Find which partial solution(s) applies to the original problem.

Usually, $S[n]$ with $n$ the size of the problem is a sub-problem equivalent to the original problem.

## Example: Shortest path (Bellman-Ford)

Find the shortest path from a node to all other nodes.

### 1. Find sub-problems

Define a sub-problem $S[u,v]$ as distance of the shortest path between nodes $u$ and $v$.

### 2. "Guess" partial solution

If we know the optimal first step, we can divide the shortest path between $u$ and $v$ as a path between $u$ and $w$, and a single optimal path between $w$ and $v$:

$$S[u,v] = \min_{w}S[u,w] + c(w,v)$$

But we have a problem here, as this definition fails for graphs with cycles.

### 1. Sub-problem definition

Define the sub-problem $S[u,v,k]$ as distance of the shortest path between nodes $u$ and $v$ using at most $k$ nodes.

### 2. "Guess" partial solution

Make explicit the length of the path:

$$S[u,v,k] = S[u,w,k-1] + c(w,v)$$

### 3. Relate sub-problems

Relate sub-problems of increasing path-length. Note that now we must compare the paths that use $k$ nodes and the paths using $k-1$ nodes:

$$S[u,v,k] = \min\left[S[u,v,k-1], \left(\min_{w}S[u,w,k-1]+c(w,v)\right)\right]$$

With

$$S[u,v,1] = c(u,v)$$

### 4. Recursion and memoization, or bottom-up approach.

#### 4.1 Memoization

```Python
G = {
    0: {1: 1, {2: 1}, ...},
    ...
}
memory = dict()
def short_path_memory(G, u, v, k):
    if k == 1:
        return G[u][v]
    if (u, v, k) in memory:
        return memory[(u, v, k)]
    Sk = min(short_path_memory(G, u, v, k-1),
             min(short_path_memory(G, u, w, k-1) + G[w][v] for w in G))
    memory[(u, v, k)] = Sk
    return Sk
```

#### 4.2 Bottom-up

```Python
INF = 1E6
def short_path_bottomup(G, u):
    D1 = defaultdict(lambda: math.inf)
    for v in G[u]:
        D1[v] = G[u][v]
    for k in range(1, len(G)):
        D2 = defaultdict(lambda: math.inf)
        for v in G:
            D2[v] = min(D1[v], min(D1[w] + G[w][v] for w in G))
        D1 = D2
    return D1
```

## Example: Edit distance

Given two strings $s_1$ and $s_2$, what is the minimum number of insertion or deletions to make both strings equal?

- Sub-problem: Calculate edit distance between $s_1[i:]$ and $s_2[j:]$
- Guess solution: If $s_1[i] \neq s_2[j]$
    - Remove $s_1[i]$ (or prepend $s_1[i]$ to $s_2[j:]$) and calculate edit distance between $s_1[i+1 :]$ and $s_2[j]$
    - Remove $s_2[j]$ (or prepend $s_2[j]$ to $s_1[i:]$) and calculate edit distance between $s_2[j+1 :]$ and $s_1[i]$
- Relate sub-problems:
$$d(s_1,s_2) = \begin{cases}
    \max|s_1|,|s_2| & |s_1|=0 \vee |s_2| = 0 \\
    d(s_1[1:],s2[1:]) & s_1[0] = s_2[0] \\
    1 + \min d(s_1[1:],s2), d(s_1,s_2[1:]) & \text{in other cases.}
\end{cases}$$
- Recursion with memoization

In [1]:
import functools
@functools.lru_cache(None)
def string_distance(s1, s2):
    if len(s1) == 0 or len(s2) == 0:
        return max(len(s1), len(s2))
    if s1[0] == s2[0]:
        return string_distance(s1[1:], s2[1:])
    else:
        return 1 + min(string_distance(s1[1:], s2),
                       string_distance(s1, s2[1:]))

In [ ]:
string_distance('electroencefalografista', 'esternocleidomastoideo')

## Classical Dynamic Programing

There are a number of classical Dynamic Programing problems for which the solution is well known:

### Max 1/2D sums

Finding the maximum sum of a 1D array can be computed by brute force in $O(n^3)$ (calculate all possible sums of all possible intervals and find the maximum one). Using `MAXSUM(i)` as the value for the maximum sum ending at index `i`, we can calculate it in $O(n)$:

```
MAXSUM(0) = V(0)
MAXSUM(i) = max(MAXSUM(i-1) + V[i], V[i])
```

And the maximum sub-sum is `max(MAXSUM(i))`.

For 2D arrays, the brute force approach can be achieved in $O(n^6)$, but by pre-computing the accumulated array, finding the maximum sub-rectangle sum can be achieved in $O(n^4)$.

### Longest increasing subsequence

The problem is to find a subsequence of numbers (not necessarily contiguous) such as the values are in an increasing order. A brute force algorithm that lists/enumerates all subsequences and then finds the longest one, has complexity $O(2^n)$. A better approach is to define the function `LIS(i)` as the longest subsequence up to index `i`, and then the recursion:

```
LIS(0) = 1
LIS(i) = max(LIS(j) + 1 for j in range(i) if seq[j] < seq[i])
```

### 0-1 Knapsack (Subset Sum)

This is a known NP problem, and as such, hard to solve. Given `n` items with values `v[i]` and weights `w[i]` and a knapsack with size `S`, find the maximum value you can carry by either taking or ignoring an item.

To solve this problem, we define `value(id, weight)` as the value of using items labeled `id` or above having remaining weight `weight`. It is possible to write the recursion as follows:

```
val(_, 0) = 0
val(N, 0) = 0
val(id, weight) = val(id + 1, weight) if w[id] > weight else
    max(val(id + 1), v[id] + + val(id + 1, weight - w[id]))
```

### Coin Change

Given a value `V` and coin values `values[c]` for coin `c`, what is the minimum number of coins we must use to represent `V`? We can represent this problem by the simple function:

```
change(0) = 0
change(v) = -inf if v < 0 else 1 + min(change(v-c) for c in values)
```